# 1NN своими руками против RandomForest

### Задание

В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.  
Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

**Задание 1**

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.  

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).  

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.  

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.  

**Задание 2**

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

How to submit
When you're ready to submit, you can upload files for each part of the assignment on the "My submission" tab.

**Библиотеки**

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets, ensemble, neighbors

**Датасет**

In [2]:
digits = datasets.load_digits()

In [3]:
# собираем в единый датафрейм
df = pd.DataFrame(digits.data).join(pd.DataFrame(digits.target, columns=['target']))
df.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,target
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0,1
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0,2
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0,3
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0,4


In [4]:
df.shape

(1797, 65)

**Делим на train/test**

In [5]:
df_train = df.iloc[:int(0.75 * df.shape[0]), :]
df_test = df.iloc[int(0.75 * df.shape[0]):, :]

**Считаем расстояние между тестовой точкой и всеми обучающими, находим минимальное расстояние, присваиваем соответствующий класс.**

In [6]:
correct = 0
for i in range(df_test.shape[0]):
    distances = np.array([np.sum((df_test.iloc[i,:-1] - df_train.iloc[j,:-1])**2) for j in range(df_train.shape[0])])
    if df_train.iloc[np.argmin(distances), -1] == df_test.iloc[i, -1]:
        correct += 1
print('Ошибка классификации:', 1 - correct/df_test.shape[0])

Ошибка классификации: 0.0377777777777778


In [7]:
def write_answer(i, answer):
    name = 'ass2_problem{}.txt'.format(i)
    with open(name, 'w') as fout:
        fout.write(str(answer))

In [8]:
# ответ 1
write_answer(1, 1 - correct/df_test.shape[0])

Обратите внимание - ошибка составляет менее 5%! Это показательный пример, что никогда не стоит забывать о бейзлайнах - иногда легко податься соблазну запустить несколько сложных алгоритмов, получить в них качество 90+%, и верить в то, что задача решена хорошо. При этом же запросто может оказаться, что простые методы дают еще лучший результат.

**Сверим ответ со sklearn.KNeighborsClassifier**

In [9]:
KNN1 = neighbors.KNeighborsClassifier(n_neighbors=1)
KNN1.fit(df_train.iloc[:,:-1], df_train.iloc[:,-1])
y_KNN1 = KNN1.predict(df_test.iloc[:,:-1])
error_KNN1 = 1 - (np.sum(df_test['target'] == y_KNN1))/df_test.shape[0]
error_KNN1

0.0377777777777778

Совпадает!

**Сравним результат с RandomForestClassifier(n_estimators=1000)**

In [10]:
RFC1000 = ensemble.RandomForestClassifier(n_estimators=1000)
RFC1000.fit(df_train.iloc[:,:-1], df_train.iloc[:,-1])
y_RFC = RFC1000.predict(df_test.iloc[:,:-1])
error_RFC = 1 - (np.sum(df_test['target'] == y_RFC))/df_test.shape[0]
error_RFC

0.0755555555555556

Обратите внимание, что качество заметно хуже, чем у 1NN. Это нестандартная ситуация, как правило Random Forest отрабатывает лучше, но такие ситуации иногда встречаются.

In [11]:
# ответ 2
write_answer(2, error_RFC)